## Data Assessment of Waterflow Historical Data

**Metadata Summary**  
- 📅 **Date of Retrieval:** JULY 1, 2025  
- 🌐 **Source of Data:** LGU San Jacinto Treasury Records
- 📄 **License/Permission:**  
- 🧑‍💼 **Prepared by:** MARK JUNE E. ALMOJUELA

In [7]:
# Initialization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [14]:
# Dataset Availability and Compatibility Check

data_dir_path = "../../dataset/raw/"
try:
    years = [year for year in os.listdir(data_dir_path) if os.path.isdir(os.path.join(data_dir_path, year))]
    print("Available years: ", years, '\n')

    # List xlsx files in the directory
    for year in years:
        xlsx_files = [f for f in os.listdir(f'{data_dir_path}{year}') if f.endswith(".csv")]
        print(f"{year}: {xlsx_files}; \nCOUNT: {len(xlsx_files)}")
except FileNotFoundError:
    print("Directory not found")
except NotADirectoryError:
    print("Path is not a directory")
except PermissionError:
    print("Permission denied")
except Exception as e:
    print(f"An error occurred: {e}")

Available years:  ['2020', '2021', '2022', '2023', '2024', '2025'] 

2020: ['FEB2020.csv', 'JAN2020.csv']; 
COUNT: 2
2021: []; 
COUNT: 0
2022: []; 
COUNT: 0
2023: []; 
COUNT: 0
2024: []; 
COUNT: 0
2025: []; 
COUNT: 0


In [ ]:
# Dataset Ingestion
data_dir_path = "../../dataset/raw/"